In [26]:
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
from urllib.request import urlopen
import urllib
import requests

import gspread
from gspread_dataframe import set_with_dataframe
import gspread_pandas
from oauth2client.client import GoogleCredentials
from google.oauth2.service_account import Credentials

import pandas as pd
import numpy as np
pd.set_option('mode.chained_assignment', None)
import json
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, date
from dateutil.relativedelta import *
import time

import mysql.connector

scopes = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
credentials = Credentials.from_service_account_file( r'/Users/sophiechance/Documents/Work Folder/GitHub/hubspot-audits/Grow_Data-ce81c41081c9.json',scopes=scopes)
gc = gspread.authorize(credentials)

pd.set_option('display.max_columns', None)
# pd.set_option("max_rows", None)

In [27]:
raw_company_df = pd.read_csv('all-companies 3.csv')
company_df = raw_company_df

/Users/sophiechance/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (8,15,20,21,24,41,57,76,77,85,102,105,107,108,110,127,129,136,146,149,171) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [42]:
len(raw_company_df)

12562

In [35]:
company_df = company_df.reindex(['Company ID',  'Company name', 'Total Students', 'School Type (picklist)', 'Company owner', 'SDR Owner', 'Segment', 'ICP?', 'Record Type', 'Carnegie Classification', 'City', '# of Campuses', 'Client Accreditor(s)',  'Website URL',  'Corporate Tier', 'Cost of Attendance', 'Full Address', 'State/Region', 'Country/Region','Graduation Rate', 'IPEDS ID', 'IPEDS link', 'LMS (multi-select)', 'SIS (multi-select)', 'School Closed', 'Send to CS'], axis=1)

In [36]:
company_df

,Company ID,Company name,Total Students,School Type (picklist),Company owner,SDR Owner,Segment,ICP?,Record Type,Carnegie Classification,City,# of Campuses,Client Accreditor(s),Website URL,Corporate Tier,Cost of Attendance,Full Address,State/Region,Country/Region,Graduation Rate,IPEDS ID,IPEDS link,LMS (multi-select),SIS (multi-select),School Closed,Send to CS
0,4108651763,St. Francis De Sales School,NaN,NaN,Evan Buechler,Falisha Fehoko,NaN,ICP,NaN,NaN,Newark,NaN,NaN,https://www.wrps.org/district/googleapps.cfm,NaN,NaN,NaN,OH,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4108677698,160 Driving Academy,20000.0,Private for-profit,Evan Buechler,Falisha Fehoko,Career,ICP,NaN,NaN,Bensenville,79.0,accsc,californiacareerschool.edu,NaN,NaN,"1100 TECHNOLOGY CIRCLE, Anaheim, CA 92805, Uni...",California,United States,NaN,NaN,NaN,Canvas,NaN,NaN,NaN
2,4108671201,"1st Choice Career Institute, Inc.",NaN,NaN,Jackson Batchelder,Joel Miller,NaN,ICP,NaN,NaN,Brooklyn,NaN,NaN,http://1stchoiceinstitute.com/,NaN,NaN,NaN,NY,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4108648042,A T Still University of Health Sciences - Kirk...,4267.0,Not-for-profit,Jackson Batchelder,Jack Strader,Career,ICP,Lone Campus,Special Focus Four-Year: Medical Schools & Cen...,Kirksville,1.0,NaN,atsu.edu/,A,NaN,"800 W Jefferson Kirksville, MO",MO,United States,NaN,177834.0,https://nces.ed.gov/ipeds/datacenter/instituti...,Canvas,NaN,NaN,NaN
4,4108744253,A-Technical College,NaN,NaN,Luke Sophinos,NaN,NaN,ICP,NaN,NaN,Huntington Park,NaN,NaN,atechcollege.edu,NaN,NaN,NaN,CA,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7622,6994320502,skpd.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,skpd.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7623,6993620999,socems.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Deerfield,NaN,NaN,socems.org,NaN,NaN,NaN,Massachusetts,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7624,7253581088,ttnpro.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ttnpro.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7625,6993629565,ucems.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lafayette,NaN,NaN,ucems.org,NaN,NaN,NaN,TN,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
company_df = company_df[company_df['Segment'] != 'Higher Education']
company_df = company_df[company_df['ICP?']!= 'Non-ICP']
company_df = company_df[company_df['ICP?']!= 'School Closed']
company_df = company_df[company_df['Send to CS'].isna()]

In [38]:
company_df.sort_values("Company name", inplace=True)
company_df.drop_duplicates(subset="Company name", keep=False, inplace=True)

company_df = company_df.reset_index().drop(['index'], axis=1)

In [39]:
company_df

,Company ID,Company name,Total Students,School Type (picklist),Company owner,SDR Owner,Segment,ICP?,Record Type,Carnegie Classification,City,# of Campuses,Client Accreditor(s),Website URL,Corporate Tier,Cost of Attendance,Full Address,State/Region,Country/Region,Graduation Rate,IPEDS ID,IPEDS link,LMS (multi-select),SIS (multi-select),School Closed,Send to CS
0,4108651763,St. Francis De Sales School,NaN,NaN,Evan Buechler,Falisha Fehoko,NaN,ICP,NaN,NaN,Newark,NaN,NaN,https://www.wrps.org/district/googleapps.cfm,NaN,NaN,NaN,OH,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4108677698,160 Driving Academy,20000.0,Private for-profit,Evan Buechler,Falisha Fehoko,Career,ICP,NaN,NaN,Bensenville,79.0,accsc,californiacareerschool.edu,NaN,NaN,"1100 TECHNOLOGY CIRCLE, Anaheim, CA 92805, Uni...",California,United States,NaN,NaN,NaN,Canvas,NaN,NaN,NaN
2,4108671201,"1st Choice Career Institute, Inc.",NaN,NaN,Jackson Batchelder,Joel Miller,NaN,ICP,NaN,NaN,Brooklyn,NaN,NaN,http://1stchoiceinstitute.com/,NaN,NaN,NaN,NY,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4108648042,A T Still University of Health Sciences - Kirk...,4267.0,Not-for-profit,Jackson Batchelder,Jack Strader,Career,ICP,Lone Campus,Special Focus Four-Year: Medical Schools & Cen...,Kirksville,1.0,NaN,atsu.edu/,A,NaN,"800 W Jefferson Kirksville, MO",MO,United States,NaN,177834.0,https://nces.ed.gov/ipeds/datacenter/instituti...,Canvas,NaN,NaN,NaN
4,4108744253,A-Technical College,NaN,NaN,Luke Sophinos,NaN,NaN,ICP,NaN,NaN,Huntington Park,NaN,NaN,atechcollege.edu,NaN,NaN,NaN,CA,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7622,6994320502,skpd.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,skpd.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7623,6993620999,socems.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Deerfield,NaN,NaN,socems.org,NaN,NaN,NaN,Massachusetts,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7624,7253581088,ttnpro.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ttnpro.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7625,6993629565,ucems.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lafayette,NaN,NaN,ucems.org,NaN,NaN,NaN,TN,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
non_icp_df = raw_company_df[raw_company_df['ICP?']== 'Non-ICP']
non_icp_df = non_icp_df[non_icp_df['Segment'] != 'Higher Education']
non_icp_df = non_icp_df[non_icp_df['ICP?']!= 'School Closed']
non_icp_df = non_icp_df[non_icp_df['Send to CS'].isna()]

non_icp_df = non_icp_df.reindex(['Company ID',  'Company name', 'Segment', 'Total Students', 'School Type (picklist)', 'Company owner', 'SDR Owner', 'ICP?', 'Record Type', 'Carnegie Classification', 'City', '# of Campuses', 'Client Accreditor(s)',  'Website URL',  'Corporate Tier', 'Cost of Attendance', 'Full Address', 'State/Region', 'Country/Region','Graduation Rate', 'IPEDS ID', 'IPEDS link', 'LMS (multi-select)', 'SIS (multi-select)', 'School Closed', 'Send to CS'], axis=1)

In [51]:
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Dfcc0KTvVlLV9mtodihPixLJe2bIhmXywi8vRxuEXMw/edit#gid=937039942')
sheet = wb.worksheet('Sheet6')

wb2 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1F0zsy6ok-uieecuURLkI59PMuT3A8KsYz7YNxt4P14s/edit#gid=258137386')
non_icp_sheet = wb2.worksheet('Non-ICP')

In [41]:
# sheet.clear()
# set_with_dataframe(sheet, company_df)

In [52]:
non_icp_sheet.clear()
set_with_dataframe(non_icp_sheet, non_icp_df)